# Importing libraries

In [1]:
import os
import random
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.6.4


# Defining variables

In [2]:
dataset_dir = '/kaggle/input/homework1/training_data_final'
submodels_dir = '/kaggle/input/ann_homework1_ensemble'

In [3]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
tf.compat.v1.set_random_seed(seed)

In [5]:
labels = ['Species1', # 1
          'Species2', # 2
          'Species3', # 3
          'Species4', # 4
          'Species5', # 5
          'Species6', # 6
          'Species7', # 7
          'Species8', # 8
]

In [6]:
image_size_2D = (96, 96)
input_shape = (96, 96, 3)
val_split = 0.2

epochs = 200
batch_size = 8
lr=1e-4

dense_freeze = 0
convnext_freeze = 0

In [7]:
model_names = ['DenseNet', 'ConvNeXt']

# Evaluation

### Performance

In [8]:
def evaluate_classes_performance(model, validation_dataset):

    data_list = []
    label_list = []
    batch_index = 0

    while batch_index <= validation_dataset.batch_index:
        data = validation_dataset.next()

        for i in range(len(data[0])):
            data_list.append(data[0][i])
            label_list.append(data[1][i])

        batch_index = batch_index + 1

    data_array = np.array(data_list)
    label_array = np.array(label_list)   
    label_values = np.argmax(label_array, axis=1)
    predictions = tl_model.predict(data_array)
    predictions = np.argmax(predictions, axis=1)
    
    report = classification_report(label_values, predictions)
    print(report)



### Plots

In [9]:
def plot_acc_loss(history):
    plt.figure(figsize=(20,5))
    plt.plot(history['loss'], label='Training', alpha=.8, color='#ff7f0e')
    plt.plot(history['val_loss'], label='Validation', alpha=.8, color='#4D61E2')
    plt.legend(loc='upper left')
    plt.title('Binary Crossentropy')
    plt.grid(alpha=.3)

    plt.figure(figsize=(20,5))
    plt.plot(history['accuracy'], label='Training', alpha=.8, color='#ff7f0e')
    plt.plot(history['val_accuracy'], label='Validation', alpha=.8, color='#4D61E2')
    plt.legend(loc='upper left')
    plt.title('Accuracy')
    plt.grid(alpha=.3)

    plt.show()

In [10]:
 def plot_double_acc_loss(history1, history2):
    plt.figure(figsize=(15,5))
    plt.plot(history1['loss'], label='Training 1', alpha=.3, color='#4D61E2', linestyle='--')
    plt.plot(history1['val_loss'], label='Validation 1', alpha=.8, color='#4D61E2')
    plt.plot(history2['loss'],  label='Training 2', alpha=.3, color='#2ABC3D', linestyle='--')
    plt.plot(history2['val_loss'], label='Validation 2', alpha=.8, color='#2ABC3D')
    plt.legend(loc='upper left')
    plt.title('Categorical Crossentropy')
    plt.grid(alpha=.3)

    plt.figure(figsize=(15,5))
    plt.plot(history1['accuracy'], label='Training 1', alpha=.3, color='#4D61E2', linestyle='--')
    plt.plot(history1['val_accuracy'], label='Validation 1', alpha=.8, color='#4D61E2')
    plt.plot(history2['accuracy'], label='Training 2', alpha=.3, color='#2ABC3D', linestyle='--')
    plt.plot(history2['val_accuracy'], label='Validation 2', alpha=.8, color='#2ABC3D')
    plt.legend(loc='upper left')
    plt.title('Accuracy')
    plt.grid(alpha=.3)

    plt.show()

# Importing the dataset

In [11]:
# Augmented ImageDataGenerator
aug_train_data_gen = ImageDataGenerator(rotation_range=20, 
                                        fill_mode='reflect',
                                        height_shift_range=20,
                                        width_shift_range=10,
                                        brightness_range=[0.7,1.2],
                                        zoom_range=0.3,
                                        vertical_flip=True,
                                        horizontal_flip=True,
                                        validation_split=val_split
                                       )
# Non-augmented ImageDataGenerator
no_aug_data_gen = ImageDataGenerator(validation_split=val_split)

In [12]:
# Load training data (augmented and non-augmented) and validation data
aug_train_gen = aug_train_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=image_size_2D,
                                               color_mode='rgb',
                                               classes=labels,
                                               subset="training",
                                               class_mode='categorical',
                                               batch_size=batch_size,
                                               shuffle=True,
                                               seed=seed)
train_gen = no_aug_data_gen.flow_from_directory(directory=dataset_dir,
                                                target_size=image_size_2D,
                                                color_mode='rgb',
                                                classes=labels,
                                                subset="training",
                                                class_mode='categorical',
                                                batch_size=batch_size,
                                                shuffle=True,
                                                seed=seed)
valid_gen = no_aug_data_gen.flow_from_directory(directory=dataset_dir,
                                                target_size=image_size_2D,
                                                color_mode='rgb',
                                                classes=labels,
                                                subset="validation",
                                                class_mode='categorical',
                                                batch_size=batch_size,
                                                shuffle=False,
                                                seed=seed)

Found 2836 images belonging to 8 classes.
Found 2836 images belonging to 8 classes.
Found 706 images belonging to 8 classes.


## General purpose classifier

### Model building

In [13]:
def compile_model(model, learning_rate):
    model.compile(
        loss=tfk.losses.CategoricalCrossentropy(),
        optimizer=tfk.optimizers.Adam(learning_rate),
        metrics='accuracy'
    )
    return model

In [14]:
def build_classifier_on_top(supernet, preprocessing_layer, name='model'):
    # Input layer
    inputs = tfk.Input(shape=input_shape, name='input')
    # Preprocessing layer
    x = preprocessing_layer(inputs)
    # Supernet 
    x = supernet(x)
    # GAP layer
    x = tfkl.GlobalAveragePooling2D(name='GAP')(x)
    # First hidden block
    x = tfkl.Dropout(0.3, name='dropout_1', seed=seed)(x)
    x = tfkl.Dense(
        256, 
        activation='relu',
        kernel_initializer = tfk.initializers.HeUniform(seed),
        name='dense_1',
    )(x)
    # Second hidden block
    x = tfkl.Dropout(0.3, name='dropout_2',seed=seed)(x)
    x = tfkl.Dense(
        256, 
        activation='relu',
        kernel_initializer = tfk.initializers.HeUniform(seed),
        name='dense_2',
    )(x)
    # Output block
    x = tfkl.Dropout(0.3, name='dropout_3',seed=seed)(x)
    outputs = tfkl.Dense(
        8, 
        activation='softmax',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        name='output',
    )(x)
    # Create and compile full model
    model = tfk.Model(inputs=inputs, outputs=outputs, name=name)
    model = compile_model(model, lr)
    
    return model

### Callbacks

In [15]:
def define_callbacks():
    callbacks = []
    # Early stopping callback
    es_callback = tfk.callbacks.EarlyStopping(
        monitor='val_accuracy',
        mode='max',
        patience=10,
        restore_best_weights=True,
    )
    callbacks.append(es_callback)
    # Other callbacks ...
    return callbacks

# DenseNet Model

### Model creation

In [16]:
# Import DenseNet model
dense_supernet = tfk.applications.DenseNet201(
    include_top=False, # Do not include classifier
    weights="imagenet",
    input_shape=input_shape
)

# Set all the layers of the net as trainable
dense_supernet.trainable = True

2022-11-27 17:13:25.713038: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [17]:
# Freeze layers
for i, layer in enumerate(dense_supernet.layers[:dense_freeze]):
    layer.trainable=False

for i, layer in enumerate(dense_supernet.layers):
    print(i, layer.name, layer.trainable)

0 input_1 True
1 zero_padding2d True
2 conv1/conv True
3 conv1/bn True
4 conv1/relu True
5 zero_padding2d_1 True
6 pool1 True
7 conv2_block1_0_bn True
8 conv2_block1_0_relu True
9 conv2_block1_1_conv True
10 conv2_block1_1_bn True
11 conv2_block1_1_relu True
12 conv2_block1_2_conv True
13 conv2_block1_concat True
14 conv2_block2_0_bn True
15 conv2_block2_0_relu True
16 conv2_block2_1_conv True
17 conv2_block2_1_bn True
18 conv2_block2_1_relu True
19 conv2_block2_2_conv True
20 conv2_block2_concat True
21 conv2_block3_0_bn True
22 conv2_block3_0_relu True
23 conv2_block3_1_conv True
24 conv2_block3_1_bn True
25 conv2_block3_1_relu True
26 conv2_block3_2_conv True
27 conv2_block3_concat True
28 conv2_block4_0_bn True
29 conv2_block4_0_relu True
30 conv2_block4_1_conv True
31 conv2_block4_1_bn True
32 conv2_block4_1_relu True
33 conv2_block4_2_conv True
34 conv2_block4_concat True
35 conv2_block5_0_bn True
36 conv2_block5_0_relu True
37 conv2_block5_1_conv True
38 conv2_block5_1_bn True
3

In [18]:
# Attach new classifier to densenet
dense_model = build_classifier_on_top(
    dense_supernet,
    tfk.applications.densenet.preprocess_input,
    name='dense_model',
)
dense_model.summary()

Model: "dense_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 96, 96, 3)]       0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 96, 96, 3)         0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 96, 96, 3)         0         
_________________________________________________________________
tf.math.truediv_1 (TFOpLambd (None, 96, 96, 3)         0         
_________________________________________________________________
densenet201 (Functional)     (None, 3, 3, 1920)        18321984  
_________________________________________________________________
GAP (GlobalAveragePooling2D) (None, 1920)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1920)              

### First training

In [ ]:
# Fine tune the model
dense_history_1 = dense_model.fit(
    x = aug_train_gen,
    validation_data = valid_gen,
    epochs = epochs,
    callbacks = define_callbacks(),
).history

In [ ]:
# Plot training history
plot_acc_loss(dense_history_1)

### Model editing

In [20]:
# Use the supernet with the fine-tuned weights only as feature extractor
dense_model.get_layer('densenet201').trainable = False

# Recompile the model
dense_model = compile_model(dense_model, lr)

dense_model.summary()

Model: "dense_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 96, 96, 3)]       0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 96, 96, 3)         0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 96, 96, 3)         0         
_________________________________________________________________
tf.math.truediv_1 (TFOpLambd (None, 96, 96, 3)         0         
_________________________________________________________________
densenet201 (Functional)     (None, 3, 3, 1920)        18321984  
_________________________________________________________________
GAP (GlobalAveragePooling2D) (None, 1920)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1920)              

### Second training

In [ ]:
# Train the classifier on non-augmented training data
dense_history_2 = dense_model.fit(
    x = train_gen,
    epochs = epochs,
    validation_data = valid_gen,
    callbacks = define_callbacks(),
).history

In [ ]:
# Plot training history
plot_double_acc_loss(dense_history_1, dense_history_2)

### Performance

In [ ]:
# Evaluate the performance scores for the model on the validation dataset 
evaluate_classes_performance(dense_model, valid_gen)

# ConvNeXt Model

### Model building

In [21]:
# Import ConvNeXt model
convnext_supernet = tfk.applications.ConvNeXtTiny(
    include_top=False, # Do not include classifier
    weights="imagenet",
    input_shape=input_shape
)

# Set all the layers of the net as trainable
convnext_supernet.trainable = True

AttributeError: module 'keras.api._v2.keras.applications' has no attribute 'ConvNeXtTiny'

In [ ]:
# Freeze layers
for i, layer in enumerate(convnext_supernet.layers[:convnext_freeze]):
    layer.trainable=False

for i, layer in enumerate(convnext_supernet.layers):
    print(i, layer.name, layer.trainable)

In [ ]:
# Attach new classifier to convnext
convnext_model = build_classifier_on_top(
    convnext_supernet,
    tfk.applications.convnext.preprocess_input,
    name='convnext_model',
)
convnext_model.summary()

### First training

In [ ]:
# Fine tune the model
convnext_history_1 = convnext_model.fit(
    x = aug_train_gen,
    validation_data = valid_gen,
    epochs = epochs,
    callbacks = define_callbacks(),
).history

In [ ]:
# Plot training history
plot_acc_loss(convnext_history_1)

### Model editing

In [ ]:
# Use the supernet with the fine-tuned weights only as feature extractor
convnext_model.get_layer('convnext_tiny').trainable = False

# Recompile the model
convnext_model = model_compile(convnext_model, lr)

convnext_model.summary()

### Second training

In [ ]:
# Train the classifier on non-augmented training data
convnext_history_1 = convnext_model.fit(
    x = train_gen,
    epochs = epochs,
    validation_data = valid_gen,
    callbacks = define_callbacks(),
).history

In [ ]:
# Plot training history
plot_double_acc_loss(convnext_history_1, convnext_history_2)

### Performance

In [ ]:
# Evaluate the performance scores for the model on the validation dataset 
evaluate_classes_performance(convnext_model, valid_gen)

# Ensemble Model

### Loading the submodels

In [ ]:
def load_all_models(model_names):
    all_models = []
    for model_name in model_names:
        filename = os.path.join(submodels_dir, model_name)
        model = tfk.models.load_model(filename)
        all_models.append(model)
        print('Successfully loaded submodule:', filename)
        evaluate_classes_performance(model, valid_gen)
    return all_models

In [ ]:
# Loading the submodels from existing saves
# submodels = load_all_models(model_names)

# Loading the submodels from in-notebook trained models
submodels = [dense_model, convnext_model]

In [ ]:
# Standardize model names and make them non-trainable
for i, model in enumerate(submodels):
    model._name = f"model_{i}"
    for layer in model.layers:
        layer.trainable = False

### Build ensemble model

In [ ]:
# Common input layer
input_layer = tfk.Input(shape=input_shape)
# Disaggregated submodel outputs
submodel_outputs = [model(input_layer) for model in submodels]
# Aggregate submodel outputs by concatenation
x = tfkl.concatenate(submodel_outputs)
# Hidden layer
x = tfkl.Dense(
    10, 
    activation='relu',
    kernel_initializer = tfk.initializers.HeUniform(seed),
    name='hidden_dense',
)(x)
# Output layer
ensemble_output = tfkl.Dense(
    8, 
    activation='softmax', 
    kernel_initializer = tfk.initializers.GlorotUniform(seed),
    name='ensemble_output',
)(x)
# Create and compile ensemble model
ens_model = tfk.Model(inputs=input_layer, outputs=ensemble_output, name='ens_model')
ens_model = compile_model(ens_model, learning_rate=1e-3)

In [ ]:
ens_model.summary()

### Ensemble training

In [ ]:
ens_history = ens_model.fit(
    x=aug_train_gen,
    epochs=epochs, 
    validation_data=valid_gen,
    callbacks = define_callbacks(),
).history

In [ ]:
# Plot the training
plot_acc_loss(ens_history)

### Performance

In [ ]:
evaluate_classes_performance(ens_model, valid_gen)